## Question 1

1. Assuming you work with 1 minute data, here are some questions to answer: What is the funding rate associated with a perpetual future? Create a function to calculate it using the logic described in Binance’s futures specification.

Simply put, the funding rate is the periodical cost for traders for holding onto a market-perceived favorable positions. The rationale behind it is to more or less allow the futures price to track whatever pair it is tracking. For instance, if the market is bullish, more people will be long than short futures, so the funding rate is positive and longs will have to pay shorts. In turn, this cuts into the profits for the longs, disincentivizing them to hold onto positions for indefinitely long times and thereby allow the futures to converge to the underlying. The opposite occurs for the shorts. 

For this task, I have used a csv file from data pulled from Binance instead of directly using their API. This is just for exploratory purposes. Assumptions for calculating funding rate: (1) ETHUSDT Futures, (2) Interest rate per 8 hours at 0.01%, per Binance specificiations, (3) use ballpark figures for checking instead of full accuracy (perps funding rates are calculated on a 5 second basis, here I am using 1 minute data, so there are huge inaccuracies). 

On Binance Futures, the interest rate is fixed at 0.03% daily by default, (0.01% per funding interval since funding occurs every 8 hours)

the Funding Rate is generally calculated by taking the time-weighted average across all 5,760 premium index data points for the 8-hour funding interval (60seconds/5 * 60min * 24hrs), but since I am using 1-minute data, I will only have 1440 data points for August 1st.


In [217]:
import pandas as pd
import numpy as np

# 1 min Premium Index data from ETHUSDT from Aug 1st - end of day
ETHUSDT = pd.read_csv('ETHUSDT_Data.csv')

In [218]:
# Transforming kline time so they are readable
ETHUSDT['open_time'] = pd.to_datetime(ETHUSDT['open_time'], unit='ms')
ETHUSDT['close_time'] = pd.to_datetime(ETHUSDT['close_time'], unit='ms')

ETHUSDT.head()

,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore
0,2024-08-01 00:00:00,-0.000398,-0.000210,-0.000546,-0.000464,0,2024-08-01 00:00:59.999,0,12,0,0,0
1,2024-08-01 00:01:00,-0.000486,-0.000054,-0.000486,-0.000343,0,2024-08-01 00:01:59.999,0,12,0,0,0
2,2024-08-01 00:02:00,-0.000348,0.000043,-0.000562,-0.000562,0,2024-08-01 00:02:59.999,0,12,0,0,0
3,2024-08-01 00:03:00,-0.000550,-0.000043,-0.000550,-0.000109,0,2024-08-01 00:03:59.999,0,12,0,0,0
4,2024-08-01 00:04:00,-0.000258,-0.000110,-0.000451,-0.000291,0,2024-08-01 00:04:59.999,0,12,0,0,0


In [219]:
# split into 3 periods to find each period funding rate
PeriodOne = ETHUSDT[0:len(ETHUSDT)//3]
PeriodTwo = ETHUSDT[len(ETHUSDT)//3:2*len(ETHUSDT)//3]
PeriodThree = ETHUSDT[2*len(ETHUSDT)//3:3*len(ETHUSDT)//3]

# Only use close_time and close_price
PeriodOne = PeriodOne[['close_time', 'close']]
PeriodTwo = PeriodTwo[['close_time', 'close']]
PeriodThree = PeriodThree[['close_time', 'close']]
PeriodThree.head()


,close_time,close
960,2024-08-01 16:00:59.999,-0.000636
961,2024-08-01 16:01:59.999,-0.000616
962,2024-08-01 16:02:59.999,-0.000295
963,2024-08-01 16:03:59.999,-0.000423
964,2024-08-01 16:04:59.999,-0.000216


In [220]:
#Question 1: 
# Funding Rate is calculated by taking the time-weighted average across all premium index data points for the 8-hour funding interval.
# Funding Rate (F) = Average Premium Index (P) + clamp (interest rate - Premium Index (P), 0.05%, -0.05%)
def FundingRateCalculator(PremiumIndexList, InterestRate, max, min):
    Dates = PremiumIndexList['close_time']
    PremiumIndexList = PremiumIndexList['close']
    TimePremiums = [] #time*premiums

    for n, PremiumIndex in enumerate(PremiumIndexList): #starts at 0
        TimePremiums.append((n+1)*PremiumIndex) #Finds the time*premiums for each premium time
    
    WeightedTimeAvgPremiums = []
    for i in range(len(TimePremiums)):
        WeightedTimeAvgPremiums.append(sum(TimePremiums[0:i+1])/((i+1)*(i+2)/2)) # 1*P1 + 2*P2 + 3*P3 + ... + n*Pn / n(n+1)/2
    
    FundingRates = []
    for P in WeightedTimeAvgPremiums:
        clamp = InterestRate - P
        if clamp <= min:
            clamp = min
        elif clamp >= max:
            clamp = max
        else:
            clamp = InterestRate - P
        
        FundingRates.append(P + clamp)
    
    #create fundingrates dataframe using dates and fundingrates
    FundingRates = pd.DataFrame({'Dates': Dates, 'FundingRates': FundingRates})
    return FundingRates

# Example FundingRates for a given period, PeriodOne is from 0 - 8hrs UTC, interest = 0.01%, cap = 0.05%, floor = -0.05%
FundingRatesOne = FundingRateCalculator(PeriodOne, 0.0001, 0.0005, -0.0005)
FundingRatesTwo = FundingRateCalculator(PeriodTwo, 0.0001, 0.0005, -0.0005)
FundingRatesThree = FundingRateCalculator(PeriodThree, 0.0001, 0.0005, -0.0005)
        

In [221]:
# Do some fact checking
# only problem is that funding rates are calculated every 5 seconds, but ballpark figure is OK
FundingRatesTwo.head()

,Dates,FundingRates
480,2024-08-01 08:00:59.999,0.000100
481,2024-08-01 08:01:59.999,0.000100
482,2024-08-01 08:02:59.999,0.000100
483,2024-08-01 08:03:59.999,0.000090
484,2024-08-01 08:04:59.999,0.000053


## Question 2 + Question 3

2. How would you calculate an implied funding rate (or implied interest rate) for a calendar future? Write a function to calculate it.

Let’s say I am trying to calculate the implied funding rate or otherwise interest rate for a quarterly-settled ETHUSDT futures contract. The contract I’m using is 0927 ETHUSDT Futures from Binance. 

![Funding Rate Calculation](0927Futures.png)
 
The same way I would price a normal currency forward. So Forward (ETHUSDT) = Spot*[(1+rUSD) / (1+rETH)]^(t), where t here is in days since I'm using daily figures. The reason 1+rETH is at the bottom because the spot rate is quoted as USDT / ETH, so 1 ETH = $3045.30. Daily Binance interest rate is 0.03% as specified. Plugging in the numbers where Forward = 3045.30, Spot = 3005.77, rUSD = 0.03%, and t = 55 day, the calculated implied 55-day funding rate on ETH is ~0.344%, which is a daily return of 0.00624% and a yearly return of ~2.28%. Doing a quick math check, this is very much in line with current near-zero risk on ETH yields, for example, 2.61% for stETH. 

3. How would you trade the spread between the rates implied by the calendar future and perpetual future?

I would lend ETH in stETH and short 0927 ETHUSDT Futures. I would (1) be delta-neutral, and (2) obtain a spread between the 55-day period return between stETH and the ETHUSDT futures. The 55-day return on stETH is ~0.393%, so I would be making 0.393-0.344 = 0.049% of no-risk arbitrage return, assuming the stETH returns do not move considerably. 


In [222]:
# Question 2 + 3

# Pricing a futures contract the same as a currency foward, using ETH as the underlying asset

def ImpliedCalculator(Spot, Forward, rUSD, T): # T in # of days btw, since rUSD here is daily rate
    rETH = (Spot/Forward * (1+rUSD)**(T)) - 1
    return rETH*100

Spot = 3005.77
Forward = 3045.30
rUSD = 0.0003
T = 55

rETH = ImpliedCalculator(Spot, Forward, rUSD, T)
print(f"Implied {T} Day Funding Rate for Ethereum: {rETH:.4f} %")

# assuming successful rollover at same rates
rETH_Yearly = rETH*365/55
print(f"Yearly additional return: {rETH_Yearly:.4f} %") # Makes sense


Implied 55 Day Funding Rate for Ethereum: 0.3438 %
Yearly additional return: 2.2814 %
